In [2]:
!pip install fredapi

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import pandas as pd
from fredapi import Fred

In [5]:
def monthly_mean_to_daily(df_monthly: pd.core.frame.DataFrame ) -> pd.core.frame.DataFrame:
    """
    Convert Monthly data into Daily data and impute with monthly mean prices
    """
    df_monthly['Date'] = pd.to_datetime(df_monthly[['Year', 'Month']].assign(DAY=1))
    df = df_monthly.explode('Date') # The explode() method converts each element of the specified column(s) into a row.

    # Generate a complete range of daily dates for the year for imputation
    start_date = df['Date'].min() # represents the starting point of your data
    end_date = df['Date'].max() + pd.offsets.MonthEnd(1)  # finds the maximum (or latest) date and include the last month fully
    full_date_range = pd.date_range(start=start_date, end=end_date, freq='D') # generates a fixed-frequency DatetimeIndex

    # Merge the full date range with the monthly averages to fill in all days
    df_full_date_range = pd.DataFrame(full_date_range, columns=['Date'])
    df = pd.merge(df_full_date_range, df_monthly, on='Date', how='left')
    df_daily = df.ffill(axis=0) # to fill the missing value based on last valid observation following index sequence
    return df_daily

## To import Electricity price from 2010 to 2023, and extend daily prices with monthly mean prices.

In [8]:
elec_df_monthly = pd.read_csv('ELECTRICITY.csv').iloc[:,1:]
elec_df_daily = monthly_mean_to_daily(elec_df_monthly)

print(elec_df_daily)
print(elec_df_daily.isna().sum().sort_values())

           Date    Year  Month  Electricity
0    2010-01-01  2010.0    1.0        46.29
1    2010-01-02  2010.0    1.0        46.29
2    2010-01-03  2010.0    1.0        46.29
3    2010-01-04  2010.0    1.0        46.29
4    2010-01-05  2010.0    1.0        46.29
...         ...     ...    ...          ...
5108 2023-12-27  2023.0   12.0       112.54
5109 2023-12-28  2023.0   12.0       112.54
5110 2023-12-29  2023.0   12.0       112.54
5111 2023-12-30  2023.0   12.0       112.54
5112 2023-12-31  2023.0   12.0       112.54

[5113 rows x 4 columns]
Date           0
Year           0
Month          0
Electricity    0
dtype: int64


## To import EU Gas price from 2010 to 2023, and extend daily prices with monthly mean prices.

In [10]:
apiKey = '29219060bc68b2802af8584e0f328b52'
fred = Fred(api_key=apiKey)

# Natural Gas prices in Europe per month
TTF_GAS = pd.DataFrame(fred.get_series('PNGASEUUSDM'), 
                       columns=['PNGASEUUSDM']).reset_index() 
TTF_GAS['index'] = pd.to_datetime(TTF_GAS['index'], format='%Y-%m-%d')
TTF_GAS['Year'] = TTF_GAS['index'].dt.year
TTF_GAS['Month'] = TTF_GAS['index'].dt.month
TTF_GAS = TTF_GAS.drop(['index'], axis=1)
TTF_GAS_2010_23_monthly = TTF_GAS[TTF_GAS['Year']>=2010].reset_index().drop(['index'], axis=1)

TTF_GAS_daily = monthly_mean_to_daily(TTF_GAS_2010_23_monthly)
print(TTF_GAS_daily)




           Date  PNGASEUUSDM    Year  Month
0    2010-01-01     7.590000  2010.0    1.0
1    2010-01-02     7.590000  2010.0    1.0
2    2010-01-03     7.590000  2010.0    1.0
3    2010-01-04     7.590000  2010.0    1.0
4    2010-01-05     7.590000  2010.0    1.0
...         ...          ...     ...    ...
5108 2023-12-27    11.296291  2023.0   12.0
5109 2023-12-28    11.296291  2023.0   12.0
5110 2023-12-29    11.296291  2023.0   12.0
5111 2023-12-30    11.296291  2023.0   12.0
5112 2023-12-31    11.296291  2023.0   12.0

[5113 rows x 4 columns]
